<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Neural_Architecture_Search_(NAS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import random

class SimpleNASModel(nn.Module):
    def __init__(self, layer_sizes):
        super(SimpleNASModel, self).__init__()
        layers = []
        input_size = layer_sizes[0]
        for size in layer_sizes[1:]:
            layers.append(nn.Linear(input_size, size))
            layers.append(nn.ReLU())
            input_size = size
        layers.pop()  # Remove last ReLU
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

def train_and_evaluate(model):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Generate synthetic data for example usage
    input_dim = model.network[0].in_features
    output_dim = model.network[-1].out_features
    data = torch.randn(100, input_dim)  # 100 samples with input dimension
    labels = torch.randn(100, output_dim)  # 100 labels with output dimension

    dataset = TensorDataset(data, labels)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

    # Training loop
    model.train()
    for epoch in range(5):  # Train for 5 epochs for simplicity
        for batch_data, batch_labels in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()

    return loss.item()

def random_architecture_search(input_dim, output_dim, num_layers, max_hidden_units):
    best_model = None
    best_loss = float('inf')

    for _ in range(10):  # Search over 10 random architectures
        layer_sizes = [input_dim] + [random.randint(1, max_hidden_units) for _ in range(num_layers)] + [output_dim]
        model = SimpleNASModel(layer_sizes)

        loss = train_and_evaluate(model)

        if loss < best_loss:
            best_loss = loss
            best_model = model

    return best_model

# Example usage
best_model = random_architecture_search(input_dim=64, output_dim=10, num_layers=3, max_hidden_units=128)